In [1]:
!gcloud auth login

/bin/sh: line 1: gcloud: command not found


In [2]:
# Set configuration values

PROJECT_ID = "andrewcooley-test-project"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://andrewcooley-reasoning-engine"

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

ModuleNotFoundError: No module named 'vertexai'

In [ ]:
# Import libraries

import requests
from googleapiclient import discovery
from IPython.display import display, Markdown
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain_core import prompts
from langchain.memory import ChatMessageHistory
from vertexai.preview import reasoning_engines

In [ ]:
# Set model api for agent to use

model = "gemini-1.5-flash-001"

In [ ]:
# Define function to retrieve review from Google Maps Places API

def google_maps_reviews(user_query: str) -> str:
    """
    Text-based search on Google Maps for a place of interest. Take a user message and create a user_query string as input to search on Google Maps for reviews of a place.
    User_query can simply be the name of a unique place or include a full address. The returned output is a string of combined reviews of the place of interest.

    Args:
        user_query (str): the text terms for a search on Google Maps to find a place id and connect to a listing of reviews.

    Returns:
        str: the reviews for the place of interest on Google Maps in the form of one combined string.
    """
    import requests

    API_KEY = ""
    BASE_URL = "https://places.googleapis.com/v1/places:searchText"

    data = {
        "textQuery": f"{user_query}"
    }

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": "places.id,places.displayName"
    }

    response = requests.post(BASE_URL, json=data, headers=headers)
    results = response.json()
    id = results.get("places", [])[0]['id']

    BASE_URL = f"https://places.googleapis.com/v1/places/{id}"

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": "id,displayName,rating,reviews"
    }
    response = requests.get(BASE_URL, headers=headers)

    combined_reviews = ""

    results = response.json()
    for review in results.get("reviews"):
        combined_reviews += review.get("text")['text']

    return combined_reviews

In [ ]:
# Test function

google_maps_reviews("McDonald's 4500 W University Dr, Prosper, TX 75034")

In [ ]:
# Define prompt template

prompt = {
    "history": lambda x: x["history"],
    "input": lambda x: x["input"],
    "agent_scratchpad": (
        lambda x: format_to_openai_function_messages(x["intermediate_steps"])
    ),
} | prompts.ChatPromptTemplate.from_messages(
    [
        ("system",
        """
        <mission>
        Your goal is to provide positive highlights about food items, given a list of customer reviews.
        These highlights will be displayed in a delivery service app.
        Follow all of your <instructions>.
        </mission>

        <instructions>
        From 'user' input, extract the relevant place and location information to perform a search.
        The search results will return reviews for a place, such as a restaurant.

        Focus only on the food items mentioned in reviews.
        Focus only on positive highlights from the reviewss.
        Disregard comments about the environment at the restaurant.
        Disregard comments about physical attractions at the restaurant.
        Disregard comments about the service at the restaurant.
        Finally, summarize the reviews.

        If there are mostly negative reviews, simply respond by saying: "There are currently not enough reviews to provide highlights."

        Do not mention delivery services in your response, such as Uber Eats.
        </instructions>
        """),
        ("placeholder", "{history}"),
        ("user", "{input}"),
        ("placeholder", "{agent_scratchpad}"),

    ]
)

# Initialize session history
store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
# Configure Reasoning Engine agent with built-in template for LangChain

agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=model,
    chat_history=get_session_history,
    model_kwargs={"temperature": 0},
    tools=[google_maps_reviews],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [3]:
# Query the local agent and view full response output

response = agent.query(
    input="""
    Tell me about Hana Hibachi and Sushi in Prosper""",
    config={"configurable": {"session_id": "new"}},
)

response

NameError: name 'agent' is not defined

In [4]:
# Display only the markdown of the agent's latest output

display(Markdown(response["output"]))

NameError: name 'Markdown' is not defined

In [ ]:
# Create remote agent that runs behind Reasoning Engine APIs

remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform",
        "langchain==0.1.20",
        "langchain-google-vertexai==1.0.3",
        "cloudpickle==3.0.0",
        "pydantic==2.7.1",
        "langchain_google_community",
    ],
)

In [ ]:
# Retrieve the project number associated with your project ID

service = discovery.build("cloudresourcemanager", "v1")
request = service.projects().get(projectId=PROJECT_ID)
response = request.execute()
project_number = response["projectNumber"]
project_number

In [ ]:
# Add a new role binding to the IAM policy

!gcloud projects add-iam-policy-binding {PROJECT_ID} \
    --member=serviceAccount:service-{project_number}@gcp-sa-aiplatform-re.iam.gserviceaccount.com \
    --role=roles/discoveryengine.editor

In [ ]:
# Test remote agent and display markdown of latest agent response output

response = remote_agent.query(
    input="Hana Hibachi and Sushi Prosper, TX",
    config={"configurable": {"session_id": "demo_11"}},
)

display(Markdown(response["output"]))